In [135]:
import requests
from bs4 import BeautifulSoup
import re
import lxml
from datetime import datetime, timedelta
from shapely.geometry import box
from fiona.drvsupport import supported_drivers
import geopandas as gpd
import rasterio as rio
from rasterio.windows import from_bounds
from rasterio.plot import show
from pathlib import Path
import numpy as np

- Input a date range and a polygon for area(s) of interest
- Find folders that are within that date range
- Find images that intersect the polygons
- Clip the images that intersect by the polygons and save geotiff of the interecting area

In [95]:
base_url = "https://data.ceda.ac.uk/neodc/sentinel_ard/data/sentinel_2"

In [96]:
start_date = "2023-05-01"
end_date = "2023-08-10"

In [97]:
def filter_sentinel2_tiles(aoi, tiles_layer=None):
    if tiles_layer is None:
        supported_drivers["KML"] = "rw"
        tiles_layer = gpd.read_file(
            "https://sentinels.copernicus.eu/documents/247904/1955685/S2A_OPER_GIP_TILPAR_MPC__20151209T095117_V20150622T000000_21000101T000000_B00.kml",
            driver="kml",
        )
    tile_list = gpd.sjoin(
        tiles_layer, aoi.to_crs(epsg=4326), how="inner", op="intersects"
    )["Name"].to_list()
    return [f"T{t}" for t in tile_list]

In [98]:
aoi = gpd.read_file("inputs/test_quarries.shp")
tile_list = filter_sentinel2_tiles(aoi)

/home/eidf072/eidf072/tomwilson-eidf072/.conda/envs/geospatial/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3548: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [99]:
def create_date_url(base_url, input_date):
    year = input_date.strftime("%Y")
    month = input_date.strftime("%m")
    day = input_date.strftime("%d")
    return f"{base_url}/{year}/{month}/{day}"


def get_existing_folders(base_url, start_date, end_date):
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")
    current_date = start_date
    urls = []
    while current_date <= end_date:
        check_url = create_date_url(base_url, current_date)
        response = requests.get(check_url, timeout=5)
        if response.status_code == 200:
            urls.append(check_url)
        current_date += timedelta(days=1)
    return urls


def extract_xml_links(url, tile_list=None):
    """Extracts XML links from the given HTML webpage URL."""
    xml_links = []
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")

        for link in soup.find_all("a", href=True):
            href = link["href"]
            if href.endswith(".xml?download=1"):
                if isinstance(tile_list, list):
                    for t in tile_list:
                        if t in href:
                            xml_links.append(href)
                else:
                    xml_links.append

    return set(xml_links)


def all_xml_list(base_url, start_date, end_date, tile_list=None):
    date_urls = get_existing_folders(base_url, start_date, end_date)
    xml_links = []
    for url in date_urls:
        xml_links.extend(extract_xml_links(url, tile_list))
    return xml_links

In [100]:
xml_links = all_xml_list(base_url, start_date, end_date, tile_list)
print(len(xml_links))
xml_links[0:2]

146


['https://dap.ceda.ac.uk/neodc/sentinel_ard/data/sentinel_2/2023/05/02/S2B_20230502_latn563lonw0021_T30VWH_ORB080_20230502121918_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref_meta.xml?download=1',
 'https://dap.ceda.ac.uk/neodc/sentinel_ard/data/sentinel_2/2023/05/02/S2B_20230502_latn581lonw0038_T30VVK_ORB080_20230502121918_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref_meta.xml?download=1']

In [101]:
def _extract_xml_cloud(xml_extract):
    supp = xml_extract.find("gmd:supplementalinformation")
    character_string = supp.find("gco:characterstring").text
    lines = character_string.split("\n")
    lines = ["".join(l.split()) for l in lines]
    for line in lines:
        if line.startswith("ARCSI_CLOUD_COVER"):
            arcsi_cloud_cover = line.split(":")[1].strip()
            val = arcsi_cloud_cover
            break
    return float(val)


def _clean_coord(coord):
    coord = coord.replace("\n", "")
    return float(coord)


def _extract_extent(xml_extract):
    minx = _clean_coord(xml_extract.find("gmd:westboundlongitude").text)
    miny = _clean_coord(xml_extract.find("gmd:southboundlatitude").text)
    maxx = _clean_coord(xml_extract.find("gmd:eastboundlongitude").text)
    maxy = _clean_coord(xml_extract.find("gmd:northboundlatitude").text)
    return box(minx, miny, maxy, maxy)


def _read_xml(url):
    # Send an HTTP GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the XML content using BeautifulSoup with lxml parser
        soup = BeautifulSoup(response.text, "lxml")
    return soup


def filter_xmls_to_gdf(xml_links, cloud_cover_max=0.4):
    retained_links = []
    retained_geom = []
    for url in xml_links:
        # read the xml
        try:
            xml_extract = _read_xml(url)
        except:
            continue
        # check if too cloudy overall
        if _extract_xml_cloud(xml_extract) > cloud_cover_max:
            continue

        # If not get extent geom and append to lists
        retained_geom.append(_extract_extent(xml_extract))
        retained_links.append(url)

    image_links = [x.replace("_meta.xml?download=1", ".tif") for x in retained_links]

    return gpd.GeoDataFrame(
        {"image_links": image_links, "geometry": retained_geom}, crs="epsg:4386"
    )


def image_links_to_aoi_gdf(aoi_gdf, xml_links):
    filtered_image_gdf = filter_xmls_to_gdf(xml_links)
    return gpd.sjoin(
        aoi_gdf,
        filtered_image_gdf.to_crs(epsg=27700),
        how="left",
        predicate="intersects",
    )

In [102]:
aoi_image_gdf = image_links_to_aoi_gdf(aoi, xml_links)

/home/eidf072/eidf072/tomwilson-eidf072/.conda/envs/geospatial/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3548: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,geometry,index_right,image_links
0,None,"POLYGON ((323473.903 935723.628, 323539.114 93...",34,https://dap.ceda.ac.uk/neodc/sentinel_ard/data...
0,None,"POLYGON ((323473.903 935723.628, 323539.114 93...",18,https://dap.ceda.ac.uk/neodc/sentinel_ard/data...
0,None,"POLYGON ((323473.903 935723.628, 323539.114 93...",23,https://dap.ceda.ac.uk/neodc/sentinel_ard/data...
0,None,"POLYGON ((323473.903 935723.628, 323539.114 93...",28,https://dap.ceda.ac.uk/neodc/sentinel_ard/data...
0,None,"POLYGON ((323473.903 935723.628, 323539.114 93...",30,https://dap.ceda.ac.uk/neodc/sentinel_ard/data...
0,None,"POLYGON ((323473.903 935723.628, 323539.114 93...",32,https://dap.ceda.ac.uk/neodc/sentinel_ard/data...
1,None,"POLYGON ((185992.814 729346.439, 186027.749 72...",22,https://dap.ceda.ac.uk/neodc/sentinel_ard/data...
1,None,"POLYGON ((185992.814 729346.439, 186027.749 72...",4,https://dap.ceda.ac.uk/neodc/sentinel_ard/data...
1,None,"POLYGON ((185992.814 729346.439, 186027.749 72...",9,https://dap.ceda.ac.uk/neodc/sentinel_ard/data...
1,None,"POLYGON ((185992.814 729346.439, 186027.749 72...",14,https://dap.ceda.ac.uk/neodc/sentinel_ard/data...


In [138]:
def read_raster_window(raster_link, gdf_row, band_idx_list=[1, 2, 3]):
    # Get the geometry bounds
    minx, miny, maxx, maxy = gdf_row.geometry.bounds
    with rio.open(raster_link) as src:
        # Create a window from the geometry
        window = from_bounds(minx, miny, maxx, maxy, src.transform)

        # Read the first three bands
        raster_data = src.read(band_idx_list, window=window)
        print(src.descriptions)
    return raster_data


def plot_sample_image(gdf):
    # make this recursive in case nodata
    gdf = gdf[gdf["image_links"].notna()]
    # Check if the GeoDataFrame is empty
    if gdf.empty:
        raise ValueError("The GeoDataFrame is empty.")

    # Select a random row
    random_row = gdf.sample(n=1).iloc[0]

    # Get the image link (assuming it's the first link in 'image_links' column)
    image_link = random_row["image_links"]

    raster_data = read_raster_window(image_link, random_row)

    show(raster_data)


def write_s2_windows_to_tif(
    gdf, output_folder="outputs", aoi_id_column=None, band_idx_list=[1, 2, 3, 7]
):
    output_folder = Path(output_folder)
    output_folder.mkdir(parents=True, exist_ok=True)
    for idx, row in gdf.iterrows():
        # Get the geometry bounds
        minx, miny, maxx, maxy = row.geometry.bounds

        # Get the image link
        image_link = row["image_links"]

        # Open the raster image
        with rio.open(image_link) as src:
            # Create a window from the geometry
            window = from_bounds(minx, miny, maxx, maxy, src.transform)

            # Read the data from the window
            window_data = src.read(band_idx_list, window=window)
            no_data_value = src.nodata

            if np.all(window_data == no_data_value) or np.all(np.isnan(window_data)):
                print(f"Skipping row {idx} as all pixels are no-data values.")
                continue

            new_transform = rio.windows.transform(window, src.transform)

            # Determine the file name
            if aoi_id_column and aoi_id_column in row:
                aoi_id = f"{row[aoi_id_column]}"
            else:
                aoi_id = idx
                # Default to row index and image file name
                image_file_name = Path(image_link).name
            file_name = output_folder / f"{aoi_id}_{image_file_name}"

            # Write to a TIFF file
            out_profile = src.profile.copy()
            out_profile.update(
                count=len(band_idx_list),
                transform=new_transform,
                width=window.width,
                height=window.height,
            )
            with rio.open(f"{file_name}.tif", "w", **out_profile) as dst:
                dst.write(window_data)
            print(f"written {file_name}")

In [139]:
write_s2_windows_to_tif(aoi_image_gdf.iloc[10:12])

written outputs/1_S2A_20230616_latn563lonw0053_T30VUH_ORB080_20230616164926_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref.tif
Skipping row 1 as all pixels are no-data values.
